導入各個套件，切詞、ROUGE、BLEU、sentence transformers、csv...等等

In [1]:
import csv, os
import jieba
from rouge_chinese import Rouge
from opencc import OpenCC
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd

c:\Serious\Program\venvs\noodle_LLM\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### B round

1. 參數選擇

In [2]:
"lora_type2-8-50E64B_B",
"lora_type2-8-100E64B_B",
"lora_type2-128-50E64B_B",
"lora_type2-128-100E64B_B",
"lora_type2-256-50E64B_B",
"lora_type2-256-100E64B_B",
"lora_type3-256-100E32B_B",
"lora_type3-256-100E64B_B_old",
"lora_type3-256-100E64B_2_B",
"lora_type3-256-100E128B_B",
"lora_type3-256-150E32B_B",
"lora_type3-256-200E32B_B",
"lora_type3-256-200E64B_B",
"lora_type3-256-200E128B_B",
"lora_type3-256-200E128B_2_B",
"lora_type3-256-300E32B_B",
"lora_type3-256-300E64B_B",
"lora_type3-256-300E128B_B",

type_list = ["type2_B", "type3_B"]
model_list = [
"lora_type3-256-300E32B_B",
"lora_type3-256-300E64B_B",
"lora_type3-256-300E128B_B",
]

In [3]:
os.chdir("C:\\Serious\\Program\\Github\\noodle_LLM")

dir_test_dataset = "./training/LLM_eval/testDataset/"
dir_chat_output= "./training/LLM_eval/chatOutput/"
dir_output = "./training/LLM_eval/evalOutput/"

# 
model_candidate = {}
for i in model_list:
    model_candidate[i] = f"{dir_chat_output}{i}.csv"

#
user_values = {}
model_values = {}
chat_len = {}

for i in type_list:
    with open(f"{dir_test_dataset}{i}.csv", mode='r', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        # init
        user_values[i] = []
        model_values[i] = []
        chat_len[i] = 0

        for row in csv_reader:
            user_values[i].append(row['user'])
            model_values[i].append(row['model'])
    if len(user_values[i]) == len(user_values[i]):
        chat_len[i] = len(user_values[i])

for i in type_list:
    print(f"{i} user values: {user_values[i]}")
    print(f"{i} model values: {model_values[i]}")
    print(f"{i} length: {chat_len[i]}")
    print()

type2_B user values: ['請問你是什麼人？', '什麼是大呷麵本家？', '我在哪裡？', '請問大呷麵本家在哪裡？', '請問大呷麵本家的電話是多少？', '請問大呷麵本家的電子郵件，我想連絡你們？', '你們是叫大甲面本家這個名字嗎？', '你們為什麼不要叫做大甲麵本家？', '大呷麵本家英文名字是什麼？', '大呷麵本家有得到什麼國際認證？', '大呷麵本家是如何製麵的？', '為什麼要用機械化產線，有哪些好處？', '請問鹽水間是幹嘛的？', '請問調製完鹽水然後要幹嘛？', '請問每次製麵時使用多少麵粉？', '麵糰混合完成的下一步是什麼？', '要怎麼製作特殊口味的麵條？', '攪拌麵團的時間怎麼確定的？', '混和好口味的麵糰下一步是什麼？', '麵餅厚度如何控制？', '要怎麼避免麵條變得太厚？', '要怎麼避免麵條變得太薄？', '把麵餅壓製好後下一步是什麼？', '麵餅要怎麼分條？', '一共有哪些規格的麵條？', '製成麵餅後用什麼工具分條？', '分條後的麵條要怎麼處理？', '麵條分條後的下一步？', '乾燥一共要多久的時間？', '為什麼需要去模擬日曬？', '請問乾燥室的工作原理？', '要怎麼確保麵條乾燥完成？', '為什麼要去模擬日曬？', '要怎麼模擬日曬來乾燥麵條？', '要怎麼判斷麵條乾燥完成？', '乾燥麵條有檢測什麼數據？', '麵條乾燥完成後下一步是什麼？', '裁切後的麵條如何分成一份？', '一份要分成100公克有什麼特殊原因嗎？', '麵條綑成一束下一步是什麼？', '什麼時候要用封口機封口後包裝？', '綑成一束的麵條要怎麼進行包裝？', '自動計量機用來做什麼？', '為什麼要用自動計量機？', '自動計量機要怎麼確保準確度？', '有哪些包裝的規格？', '請問封口包裝後下一步是什麼？', '製麵工廠如何確保產品的安全性？', '金屬檢測器的功能是什麼？', '為什麼需要用金屬檢測器？', '請問製麵的最終步驟是什麼？', '請問成品麵條的最終包裝過程是什麼？', '請問你們的包裝流程是什麼？', '請問你們如何進行成品的最終包裝？', '要怎麼確保麵條在包裝後不會受到污染？', '製麵工廠是怎麼防止成品在包裝後被污染？', '製麵工廠是怎麼防止異物進入包裝？', '請問清潔區是甚麼？',

2. 測試評估資料集是否有讀取到

In [4]:
for i in type_list:    
    messages={"role": "user", "content": user_values[i][chat_len[i]-1]}
    print(messages)

{'role': 'user', 'content': '幫我介紹綠茶麵條'}
{'role': 'user', 'content': '請問綠茶麵條價格多少？'}


3. 匯入模型問答csv

In [5]:

real_answer={}
latency_t = {}

# 動態創建變數並存儲 Model's Answer
for key, file_path in model_candidate.items():
    df = pd.read_csv(f'{file_path}')

    # 提取 "Model's Answer" 列並轉換為列表
    real_answer[key] = df["Model's Answer"].tolist()
    latency_t[key] = df["Latency"].tolist()

# 確認變數是否正確創建和存儲
for i in real_answer:
    print(f"{i}: {real_answer[i][5]}")

lora_type3-256-300E32B_B: 大呷麵本家的電話是04-2686-1026。
lora_type3-256-300E64B_B: 大呷麵本家的電話是04-2686-1026。
lora_type3-256-300E128B_B: 大呷麵本家的電話是04-2686-1026。


4. 評估 ROUGE

In [6]:
cc = OpenCC('t2s')
rouge = Rouge()
def evaluate_ROUGE(model_values, model_ans, model_name, scores_list, count):
    
    for i in range(count):
        completion = model_ans[i]
        reference = model_values[i]

        reference = cc.convert(reference)
        reference = '/ '.join(jieba.cut(reference, cut_all=False))
        
        generated_answer = completion
        generated_answer = cc.convert(generated_answer)
        generated_answer = '/ '.join(jieba.cut(generated_answer, cut_all=False))
        
        score = rouge.get_scores(generated_answer, reference)
        scores_list.append(score)
        
    avg_scores = {'rouge-1': {'r': 0, 'p': 0, 'f': 0},
        'rouge-2': {'r': 0, 'p': 0, 'f': 0},
        'rouge-l': {'r': 0, 'p': 0, 'f': 0}}

    num_instances = len(scores_list)
    for instance_scores in scores_list:
        for metric, metrics_scores in instance_scores[0].items():
            for score_type, value in metrics_scores.items():
                if score_type != 'f':  # 暫時跳過 'f' 分數
                    avg_scores[metric][score_type] += value / num_instances

    # 根據 (r + p) / 2 來計算新的 'f' 分數，並四捨五入到小數點第四位
    for metric, metrics_scores in avg_scores.items():
        r = metrics_scores['r']
        p = metrics_scores['p']
        avg_scores[metric]['f'] = round((r + p) / 2, 4)
        avg_scores[metric]['r'] = round(r, 4)
        avg_scores[metric]['p'] = round(p, 4)

    # 計算 rouge-1, rouge-2, rouge-l 的 f 分數平均值
    f_score_avg = (avg_scores['rouge-1']['f'] + avg_scores['rouge-2']['f'] + avg_scores['rouge-l']['f']) / 3
    f_score_avg = round(f_score_avg, 4)
    tmp = []

    print(model_name,"Average Scores:")
    for metric, metrics_scores in avg_scores.items():
        print(f"{metric}: {metrics_scores}")
        tmp.append(metrics_scores['f'])
    print(f"\nAverage f1-score: {f_score_avg}")
    print("---------------------------------------------------------")

   
    ori_data[model_name] = (tmp[0], tmp[1], tmp[2], f_score_avg)
    



scores_dict = {}
for i in real_answer:
    scores_dict[i] = []

ori_data = {}
if os.path.isfile(f'{dir_output}ROUGE_B.csv'):
    with open(f'{dir_output}ROUGE_B.csv', mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            ori_data[row["model_name"]] = (row["rouge-1"],row["rouge-2"],row["rouge-l"],row["Average"])

with open(f'{dir_output}ROUGE_B.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["model_name", "rouge-1", "rouge-2","rouge-l","Average"]) 
    for i in real_answer:
        if "type2" in i:
            std_answer = model_values["type2_B"]
            c_len = chat_len["type2_B"]
        elif "type3" in i:
            std_answer = model_values["type3_B"]
            c_len = chat_len["type3_B"]
        evaluate_ROUGE(std_answer, real_answer[i], i, scores_dict[i], c_len)

    for i in ori_data:
        writer.writerow([i, ori_data[i][0], ori_data[i][1], ori_data[i][2], ori_data[i][3]])
      
    

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Sean\AppData\Local\Temp\jieba.cache
Loading model cost 0.530 seconds.
Prefix dict has been built successfully.


lora_type3-256-300E32B_B Average Scores:
rouge-1: {'r': 0.7575, 'p': 0.691, 'f': 0.7243}
rouge-2: {'r': 0.6145, 'p': 0.5625, 'f': 0.5885}
rouge-l: {'r': 0.7239, 'p': 0.6504, 'f': 0.6872}

Average f1-score: 0.6667
---------------------------------------------------------
lora_type3-256-300E64B_B Average Scores:
rouge-1: {'r': 0.7292, 'p': 0.4991, 'f': 0.6141}
rouge-2: {'r': 0.5299, 'p': 0.3516, 'f': 0.4407}
rouge-l: {'r': 0.6826, 'p': 0.4155, 'f': 0.549}

Average f1-score: 0.5346
---------------------------------------------------------
lora_type3-256-300E128B_B Average Scores:
rouge-1: {'r': 0.7513, 'p': 0.6683, 'f': 0.7098}
rouge-2: {'r': 0.5925, 'p': 0.5185, 'f': 0.5555}
rouge-l: {'r': 0.7094, 'p': 0.6108, 'f': 0.6601}

Average f1-score: 0.6418
---------------------------------------------------------


5. 評估 Sentence-Transformers

In [7]:
ST_model = SentenceTransformer("distiluse-base-multilingual-cased-v2")


def evaluate_ST(model_values, model_ans, model_name, scores_dict, count):
    for i in range(0, count):
        completion = model_ans[i]
        reference = model_values[i]
        generated_answer = completion
        
        e_q = ST_model.encode(generated_answer)        
        embeddings = ST_model.encode(reference)       
        sym = float(util.pytorch_cos_sim(e_q, embeddings).detach().numpy()[0][0])
        score = sym
        scores_dict.append(score)
        
    avg_scores = round(sum(scores_dict)/len(scores_dict), 4)
    print(model_name,"SentenceTransformer Semantic Similarity score:", avg_scores)

    ori_data[model_name] = avg_scores


scores_dict = {}
for i in real_answer:
    scores_dict[i] = []

ori_data = {}
if os.path.isfile(f'{dir_output}ST_B.csv'):
    with open(f'{dir_output}ST_B.csv', mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            ori_data[row["model_name"]] = (row["ST"])

with open(f'{dir_output}ST_B.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["model_name", "ST"]) 
    for i in real_answer:
        if "type2" in i:
            std_answer = model_values["type2_B"]
            c_len = chat_len["type2_B"]
        elif "type3" in i:
            std_answer = model_values["type3_B"]
            c_len = chat_len["type3_B"]
        evaluate_ST(std_answer, real_answer[i], i, scores_dict[i], c_len)

    for i in ori_data:
        writer.writerow([i, ori_data[i]])


lora_type3-256-300E32B_B SentenceTransformer Semantic Similarity score: 0.7734
lora_type3-256-300E64B_B SentenceTransformer Semantic Similarity score: 0.6622
lora_type3-256-300E128B_B SentenceTransformer Semantic Similarity score: 0.7691


6. 評估 BLEU

In [8]:
def evaluate_BLEU(model_values, model_ans, model_name, scores_dict, count):

    
    candidate_list = list()
    reference_list = list()
    
    bleu_1_scores = []
    bleu_2_scores = []
    bleu_3_scores = []
    bleu_4_scores = []
    bleu_avg_scores = []
    
    
    
    for i in range(0, count):
        completion = model_ans[i]
        reference = model_values[i]     
        
        # 將生成的句子逐字切分
        generated_answer_sepWords = ' '.join(jieba.cut(completion))
        generated_answer = list(generated_answer_sepWords)
        candidate_list.append(generated_answer)
    
        # 將參考句子逐字切分
        reference_sepWords = ' '.join(jieba.cut(reference))
        reference_answer = list(reference_sepWords)
        reference_list.append([reference_answer])

        bleu_1_score = sentence_bleu(references=reference_list[i], hypothesis=candidate_list[i], weights=(1, 0, 0, 0))
        bleu_2_score = sentence_bleu(references=reference_list[i], hypothesis=candidate_list[i], weights=(0, 1, 0, 0))
        bleu_3_score = sentence_bleu(references=reference_list[i], hypothesis=candidate_list[i], weights=(0, 0, 1, 0))
        bleu_4_score = sentence_bleu(references=reference_list[i], hypothesis=candidate_list[i], weights=(0, 0, 0, 1))
        bleu_avg_score = sentence_bleu(references=reference_list[i], hypothesis=candidate_list[i])
        
        bleu_1_scores.append(bleu_1_score)
        bleu_2_scores.append(bleu_2_score)
        bleu_3_scores.append(bleu_3_score)
        bleu_4_scores.append(bleu_4_score)
        bleu_avg_scores.append(bleu_avg_score)
        
    avg_bleu_1 = round(sum(bleu_1_scores) / len(bleu_1_scores), 4)
    avg_bleu_2 = round(sum(bleu_2_scores) / len(bleu_2_scores), 4)
    avg_bleu_3 = round(sum(bleu_3_scores) / len(bleu_3_scores), 4)
    avg_bleu_4 = round(sum(bleu_4_scores) / len(bleu_4_scores), 4)
    avg_bleu_avg = round(sum(bleu_avg_scores) / len(bleu_avg_scores), 4)
    print(f"{model_name} BLEU-1 score: {avg_bleu_1}")
    print(f"{model_name} BLEU-2 score: {avg_bleu_2}")
    print(f"{model_name} BLEU-3 score: {avg_bleu_3}")
    print(f"{model_name} BLEU-4 score: {avg_bleu_4}")
    print(f"{model_name} BLEU score: {avg_bleu_avg}")
    print("-----------------------------------------------------")
    
    ori_data[model_name] = avg_bleu_avg


scores_dict = {}
for i in real_answer:
    scores_dict[i] = []

ori_data = {}
if os.path.isfile(f'{dir_output}BLEU_B.csv'):
    with open(f'{dir_output}BLEU_B.csv', mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            ori_data[row["model_name"]] = (row["BLEU"])

with open(f'{dir_output}BLEU_B.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["model_name", "BLEU"]) 
    for i in real_answer:
        if "type2" in i:
            std_answer = model_values["type2_B"]
            c_len = chat_len["type2_B"]
        elif "type3" in i:
            std_answer = model_values["type3_B"]
            c_len = chat_len["type3_B"]
        evaluate_BLEU(std_answer, real_answer[i], i, scores_dict[i], c_len)
    for i in ori_data:
        writer.writerow([i, ori_data[i]])



lora_type3-256-300E32B_B BLEU-1 score: 0.6879
lora_type3-256-300E32B_B BLEU-2 score: 0.6261
lora_type3-256-300E32B_B BLEU-3 score: 0.5822
lora_type3-256-300E32B_B BLEU-4 score: 0.5482
lora_type3-256-300E32B_B BLEU score: 0.6003
-----------------------------------------------------


c:\Serious\Program\venvs\noodle_LLM\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Serious\Program\venvs\noodle_LLM\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


lora_type3-256-300E64B_B BLEU-1 score: 0.457
lora_type3-256-300E64B_B BLEU-2 score: 0.4019
lora_type3-256-300E64B_B BLEU-3 score: 0.36
lora_type3-256-300E64B_B BLEU-4 score: 0.3291
lora_type3-256-300E64B_B BLEU score: 0.3747
-----------------------------------------------------
lora_type3-256-300E128B_B BLEU-1 score: 0.6517
lora_type3-256-300E128B_B BLEU-2 score: 0.5777
lora_type3-256-300E128B_B BLEU-3 score: 0.5301
lora_type3-256-300E128B_B BLEU-4 score: 0.4932
lora_type3-256-300E128B_B BLEU score: 0.5502
-----------------------------------------------------


7. 評估延遲

In [9]:
def evaluate_latency(model_name):
    average = sum(latency_t[model_name]) / len(latency_t[model_name])
    average = round(average, 3)
    print(f"{model_name} latency: {average} s")

    ori_data[model_name] = average



ori_data = {}
if os.path.isfile(f'{dir_output}LATENCY_B.csv'):
    with open(f'{dir_output}LATENCY_B.csv', mode='r', newline='', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            ori_data[row["model_name"]] = row["LATENCY"]

with open(f'{dir_output}LATENCY_B.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["model_name", "LATENCY"])
    for i in model_list:
        evaluate_latency(i)
    
    for i in ori_data:
        writer.writerow([i, ori_data[i]])

lora_type3-256-300E32B_B latency: 4.315 s
lora_type3-256-300E64B_B latency: 6.83 s
lora_type3-256-300E128B_B latency: 4.166 s
